<a href="https://colab.research.google.com/github/drpetros11111/ModernCompVision/blob/01_OpenCV/4_PyTorch_Misclassifications_and_Model_Performance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://github.com/rajeevratan84/ModernComputerVision/raw/main/logo_MCV_W.png)

# **PyTorch Model Performance Analysis**
---



---



In this lesson, we learn use the MNIST model we trained in the previously lesson and analyze it's performance, we do:
1. Setup Our PyTorch Model and Data
2. Load the previously trained model
3. View the images we misclassified
4. Create a Confusion Matrix
5. Create Classification Report


# **1. Setup our PyTorch Imports, Model and Load the MNIST Dataset**

We only need to load the Test dataset since we're analyzing performance on that data segment.

In [3]:
# Import PyTorch
import torch

# We use torchvision to get our dataset and useful image transformations
import torchvision
import torchvision.transforms as transforms

# Import PyTorch's optimization libary and nn
# nn is used as the basic building block for our Network graphs
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

# Are we using our GPU?
print("GPU available: {}".format(torch.cuda.is_available()))

# Set device to cuda
device = 'cuda'

GPU available: True


#### **Our Image plotting function**

In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Define our imshow function
def imgshow(title, image = None, size = 6):
      w, h = image.shape[0], image.shape[1]
      aspect_ratio = w/h
      plt.figure(figsize=(size * aspect_ratio,size))
      plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      plt.title(title)
      plt.show()

# Define the function  imgshow

-----------------------------
# Import necessary libraries:

#import cv2:
Imports the OpenCV library (cv2) for image processing.

##import numpy as np:
Imports the NumPy library (np) for numerical operations, particularly for working with arrays.

###from matplotlib import pyplot as plt:
Imports the pyplot module from Matplotlib (plt) for creating visualizations, especially plots and images.

------------------------------
#Define the imgshow function

    def imgshow(title, image=None, size=6):
    
This line defines a function named

##imgshow that takes three arguments:

###title:
A string representing the title of the image display.

###image:
The image data itself (presumably a NumPy array). It defaults to None if not provided.

###size:
A numerical value controlling the display size of the image (defaulting to 6).

-----------------------------------
#Calculate aspect ratio:

##w, h = image.shape[0], image.shape[1]:
These lines extract the width (w) and height (h) of the input image from its shape attribute.

##aspect_ratio = w/h:
This line calculates the aspect ratio of the image by dividing the width by the height.

---------------------------------
#Create a Matplotlib figure:

##plt.figure(figsize=(size * aspect_ratio, size)):
This line creates a new Matplotlib figure with a specific size.

##plt.figure():
This function creates a new figure window in which the image will be displayed.

A figure is like a container that holds one or more plots or images.

##figsize=(size * aspect_ratio, size):
This argument specifies the size of the figure window.

figsize is a keyword argument that takes a tuple representing the width and height of the figure in inches.

size * aspect_ratio is the calculated width of the figure.

size is the height of the figure.

Let's consider an example:

Suppose you want to display an image with an aspect ratio of 1.5 (width is 1.5 times the height), and you want the height of the displayed image to be 6 inches. Then, this line would become:

    plt.figure(figsize=(6 * 1.5, 6))

This means the figure window would be created with a width of 9 inches (6 * 1.5) and a height of 6 inches.

This ensures that the image is displayed with the correct aspect ratio, preventing distortion.

##In summary:

The line plt.figure(figsize=(size * aspect_ratio, size)) is crucial for creating a figure window with the appropriate dimensions to display the image without distortion.

It calculates the width based on the aspect ratio and desired height, providing a visually accurate representation of the image.

---------------------
#Display the image:

##plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)):
This line displays the image using Matplotlib's imshow function.

Note that OpenCV often stores images in BGR (Blue, Green, Red) format, so this line converts the image to RGB (Red, Green, Blue) format before displaying it using Matplotlib.

----------------------------
#Set the title:

##plt.title(title):

This line sets the title of the image display using the provided title argument.

--------------------------
#Show the plot:

##plt.show():

This line finally displays the Matplotlib figure containing the image and its title.

In essence, the imgshow function provides a convenient way to display images within your Colab notebook using OpenCV and Matplotlib, ensuring the image is presented with the correct aspect ratio and a title.

This helps in visualizing images during data exploration or model analysis tasks.

### **Loading our MNIST Test Dataset**

In [5]:
# Transform to a PyTorch tensors and the normalize our valeus between -1 and +1
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5, ), (0.5, )) ])

# Load our Test Data and specify what transform to use when loading
testset = torchvision.datasets.MNIST('mnist',
                                     train = False,
                                     download = True,
                                     transform = transform)

testloader = torch.utils.data.DataLoader(testset,
                                          batch_size = 128,
                                          shuffle = False,
                                          num_workers = 0)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.59MB/s]


Extracting mnist/MNIST/raw/train-images-idx3-ubyte.gz to mnist/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]


Extracting mnist/MNIST/raw/train-labels-idx1-ubyte.gz to mnist/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.09MB/s]


Extracting mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.30MB/s]

Extracting mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist/MNIST/raw



# Chain multiple image transformations together.
This function from torchvision.transforms allows you to chain multiple image transformations together.

------------------------------
---------------------------------------
##transforms.ToTensor():

This transformation converts the PIL Image (a common image format) representing each MNIST digit into a PyTorch tensor.

This is necessary for PyTorch to work with the data.

----------------------------

#1. transforms.Normalize((0.5, ), (0.5, )):
This transformation normalizes the pixel values of the image.

It subtracts the mean (0.5) from each pixel and then divides by the standard deviation (0.5).

This helps improve the performance of the model by ensuring the pixel values are within a specific range (usually between -1 and 1).

The arguments (0.5, ) and (0.5, ) represent the mean and standard deviation for each color channel (since MNIST images are grayscale, there's only one channel).

--------------------
#2. Loading the MNIST Dataset:


## Load our Test Data and specify what transform to use when loading
    testset = torchvision.datasets.MNIST
        ('mnist',
        train = False,
        download = True,
        transform = transform)

testloader = torch.utils.data.DataLoader(testset,
                                        
    batch_size = 128,
    shuffle = False,
    num_workers = 0)

    torchvision.datasets.MNIST:

This function loads the MNIST dataset.
'mnist': Specifies the directory where the dataset will be stored.

    train = False:

Indicates that we want to load the test dataset, not the training dataset.

    download = True:

Instructs the function to download the dataset if it's not already present in the specified directory.

##transform = transform:
Applies the previously defined transformation (transform) to the images as they are loaded.

##torch.utils.data.DataLoader:
This function creates a data loader that is used to iterate through the dataset in batches.

##testset:
The dataset to be loaded.

###batch_size = 128:
Specifies the number of images to be processed in each batch.

###shuffle = False:
Indicates that the data should not be shuffled.

This is important for evaluation, as we want to maintain the original order of the data.

##num_workers = 0:
Sets the number of worker processes to be used for data loading.

0 means that the data will be loaded in the main process.

--------
#In summary:

This code snippet prepares the MNIST dataset for testing a PyTorch model by:

Defining a transformation that converts images to PyTorch tensors and normalizes their pixel values.


Loading the MNIST test dataset and applying the transformation to the images.

Creating a data loader to iterate through the dataset in batches.

This process ensures that the data is in the correct format and ready to be fed into the PyTorch model for evaluation.

### **Creating our Model Defination Class**

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# **2. Loading Out Model**

I have uploaded the model to my Google Drive - https://drive.google.com/file/d/1yj01iUbYL8ZXHiYRE5Xd639tddSAkzKs/view?usp=sharing

We use gdown in our terminal to download the model file that we trained in the last lesson.

March 4th 2022 Update: File moved to S3

In [7]:
!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/mnist_cnn_net.pth

--2024-11-01 22:48:39--  https://moderncomputervision.s3.eu-west-2.amazonaws.com/mnist_cnn_net.pth
Resolving moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)... 52.95.150.66, 52.95.149.126, 52.95.149.178, ...
Connecting to moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)|52.95.150.66|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-11-01 22:48:40 ERROR 404: Not Found.



#### **NOTE**

When Loading our model we need to create the model instance i.e. ```net = Net()``` and then since we trained it using our GPU in Colab, we move it to the GPU using ```net.to(device``` where device = 'cuda'.

Then we can load our downloaded model's weights.

In [8]:
# Create an instance of the model
net = Net()
net.to(device)

# Load weights from the specified path
net.load_state_dict(torch.load('mnist_cnn_net.pth'))

<ipython-input-8-bd6ca3d081df>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('mnist_cnn_net.pth'))


FileNotFoundError: [Errno 2] No such file or directory: 'mnist_cnn_net.pth'

Model Loaded successfully if ```All keys matched successfully``` is displayed.

### **Now Let's calculate it's accuracy (done in the prevoiusly lesson so this is just a recap) on the Test Data**



In [10]:
!pip install torch torchvision torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import os

# Assuming you have a CUDA-enabled device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Create an instance of the model
net = Net()
net.to(device)

# Download the weights file and specify the file path
# Make sure this cell is executed before loading the weights
weights_path = 'mnist_cnn_net.pth'
if not os.path.exists(weights_path):
    !wget -O {weights_path} https://moderncomputervision.s3.eu-west-2.amazonaws.com/mnist_cnn_net.pth


# Load weights from the specified path
# the file should be in the current working directory
net.load_state_dict(torch.load(weights_path))

--2024-11-01 22:50:14--  https://moderncomputervision.s3.eu-west-2.amazonaws.com/mnist_cnn_net.pth
Resolving moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)... 52.95.144.2, 52.95.142.110, 52.95.150.138, ...
Connecting to moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)|52.95.144.2|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-11-01 22:50:14 ERROR 404: Not Found.



<ipython-input-10-88abb26fc1e6>:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(weights_path))


EOFError: Ran out of input

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        # Move our data to GPU
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 10000 test images: {accuracy:.3}%')

# **3. Displaying our Misclassified Images** ##

Out of 10,000 images our model predicted 98.7% correct. This is good for a first attempt with such a simple model. (there are much better models).

**A Good Practise!**

It's a good habit when creating image classifiers to visually inspect the images being mis-classified.
1. We can spot what types of images are challenging for our model
2. We can spot any incorrectly labeled images
3. If sometimes we can't correctly identify the class, seeing your CNN struggle hurts less :)

**Reminder** on why we use ```net.eval()``` and ```torch.no_grad()```

[Taken from Stackoverflow:](https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch)

**model.eval()** is a kind of switch for some specific layers/parts of the model that behave differently during training and inference (evaluating) time. For example, **Dropouts** Layers, BatchNorm Layers etc. You need to turn off them during model evaluation, and .eval() will do it for you. In addition, the common practice for evaluating/validation is using torch.no_grad() in pair with model.eval() to turn off gradients computation.

So, while we don't use Dropouts or BatchNorm in our model, it's good practice to use it when doing inference.

In [ ]:
# Set model to evaluation or inference mode
net.eval()

# We don't need gradients for validation, so wrap in
# no_grad to save memory
with torch.no_grad():
    for data in testloader:
        images, labels = data

        # Move our data to GPU
        images = images.to(device)
        labels = labels.to(device)

        # Get our outputs
        outputs = net(images)

        # use torch.argmax() to get the predictions, argmax is used for long_tensors
        predictions = torch.argmax(outputs, dim=1)

        # For test data in each batch we identify when predictions did not match the labe
        # then we print out the actual ground truth
        for i in range(data[0].shape[0]):
            pred = predictions[i].item()
            label = labels[i]
            if(label != pred):
                print(f'Actual Label: {label}, Predicted Label: {pred}')
                img = np.reshape(images[i].cpu().numpy(),[28,28])
                imgshow("", np.uint8(img), size = 1)

# **4. Creating our Confusion Matrix**

We use Sklean's Confusion Matrix tool to create it. All we need is:
1. The true labels
2. The predicted labels


In [ ]:
from sklearn.metrics import confusion_matrix


# Initialize blank tensors to store our predictions and labels lists(tensors)
pred_list = torch.zeros(0, dtype=torch.long, device='cpu')
label_list = torch.zeros(0, dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(testloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = net(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        pred_list = torch.cat([pred_list, preds.view(-1).cpu()])
        label_list = torch.cat([label_list, classes.view(-1).cpu()])

# Confusion matrix
conf_mat = confusion_matrix(label_list.numpy(), pred_list.numpy())
print(conf_mat)

#### **Interpreting the Confusion Matrix**
![](https://github.com/rajeevratan84/ModernComputerVision/raw/main/CleanShot%202020-11-30%20at%2010.46.45.png)

### **Creating a more presentable plot**

We'll reuse this nicely done function from the sklearn documentation on plotting a confusion matrix using color gradients and labels.

In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
target_names = list(range(0,10))
plot_confusion_matrix(conf_mat, target_names)

## **Let's look at our per-class accuracy**

In [ ]:
# Per-class accuracy
class_accuracy = 100 * conf_mat.diagonal() / conf_mat.sum(1)

for (i,ca) in enumerate(class_accuracy):
    print(f'Accuracy for {i} : {ca:.3f}%')

# **5. Now let's look at the Classification Report**

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(label_list.numpy(), pred_list.numpy()))

### **5.1 Support is the total sum of that class in the dataset**

### **5.2 Review of Recall**

![](https://github.com/rajeevratan84/ModernComputerVision/raw/main/CleanShot%202020-11-30%20at%2011.11.12.png)

### **5.3 Review of Precision**

![](https://github.com/rajeevratan84/ModernComputerVision/raw/main/CleanShot%202020-11-30%20at%2011.11.22.png)

### **5.4 High recall (or sensitivity) with low precision.**
This tells us that most of the positive examples are correctly recognized (low False Negatives) but there are a lot of false positives i.e. other classes being predicted as our class in question.

### **5.5 Low recall (or sensitivity) with high precision.**

Our classifier is missing a lot of positive examples (high FN) but those we predaict as positive are indeed positive (low False Positives)
